In [1]:
!{'pip install --quiet apache_beam'}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 793.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [24]:
import apache_beam as beam

class Filter_Dept(beam.DoFn):
  def process(self, element, dept):
    if element[3]==dept:
      return [element]

class Split_Row(beam.DoFn):
  def process(self, element):
    return [element.split(',')]

class PairWithOne(beam.DoFn):
  def process(self, element):
    return [(element[3] + ", "+ element[1], 1)]

class SumAllValues(beam.DoFn):
  def process(self, element):
    key, values = element
    return [(key, sum(values))]

with beam.Pipeline() as p:
  input_collection = (
      p
      | 'read from text file' >> beam.io.ReadFromText('drive/MyDrive/APACHE_BEAM/datasets/dept_data.txt')
      | 'split rows' >> beam.ParDo(Split_Row())
  )
  accounts_count = (
      input_collection
      | 'Get Accounts dept records' >> beam.ParDo(Filter_Dept(), "Accounts")
                                      #beam.Filter(lambda record: record[3]=="Accounts")
      | 'Pair Account record with 1' >> beam.ParDo(PairWithOne())

      | 'Group_Accounts' >> beam.GroupByKey()
      | 'Sum_Accounts' >> beam.ParDo(SumAllValues())
      # | 'group and sum Account' >> beam.CombinePerKey(sum)

      # | 'Write results for account' >> beam.io.Write('data/Account')
  )

  hr_count = (
      input_collection
      | 'Get HR dept records' >> beam.ParDo(Filter_Dept(), "HR")
                                  #beam.Filter(lambda record: record[3]=="HR")
      | 'Pair HR record with 1' >> beam.ParDo(PairWithOne())

      | 'Group_HR' >> beam.GroupByKey()
      | 'Sum_HR' >> beam.ParDo(SumAllValues())
      # | 'group and sum HR' >> beam.CombinePerKey(sum)

      # | 'Write results for account' >> beam.io.Write('data/Account')
  )

  final_count = (
      (accounts_count, hr_count)
      | beam.Flatten()
      |beam.Map(print)
      # | beam.io.WriteToText("drive/MyDrive/APACHE_BEAM/output/final_data.txt")
  )

('Accounts, Marco', 31)
('Accounts, Rebekah', 31)
('Accounts, Itoe', 31)
('Accounts, Edouard', 31)
('Accounts, Kyle', 62)
('Accounts, Kumiko', 31)
('Accounts, Gaston', 31)
('Accounts, Ayumi', 30)
('HR, Beryl', 62)
('HR, Olga', 31)
('HR, Leslie', 31)
('HR, Mindy', 31)
('HR, Vicky', 31)
('HR, Richard', 31)
('HR, Kirk', 31)
('HR, Kaori', 31)
('HR, Oscar', 31)
